Dave made a list of taxonomic groups that should be included in the reference database:

1. Hexapoda 1.4 million seqs
2. Arachnida 76,572 seqs
3. Myriapoda  2,124 seqs
5. Mollusca (some, complete coverage not required)
6. Nematomorpha (just to see)
6. Oniscidea (woodlice)
7. Fungi; a few ascomycetes and basidomycetes
8. Vertebrate; a few diverse bird and mammal maybe

To make the download easier we are going to break down 'Hexapoda' into smaller groups:

    collembola
    diplura
    protura
    insecta
        Ephemeroptera
        Odonata
        Embioptera
        Endopterygota
        Amphiesmenoptera
        Coleoptera
        Diptera
        Hymenoptera
        Mecoptera
        Neuropterida
        Siphonaptera
        Orthopteroidea
        Paraneoptera
        Plecoptera

Write everything to a text file and let a script do the work - process the file line by line downloading all available COI sequences for the taxa.

In [2]:
%%file hexapoda_to_download.txt
collembola
diplura
protura
Ephemeroptera
Odonata
Embioptera
Amphiesmenoptera
Coleoptera
Diptera
Hymenoptera
Mecoptera
Neuropterida
Siphonaptera
Orthopteroidea
Paraneoptera
Plecoptera

Overwriting hexapoda_to_download.txt


In [3]:
!fetch_from_db.py -h

usage: fetch_from_db.py [-h] [-t <FILE>] [-a <FILE>] [-m <string>] [-G] [-B]
                        [--no-download] [--geo <string>] [-o <string>]
                        [-@ <email-address>] [--version]

Fetch all available DNA sequences for a list of taxa and a given marker gene

optional arguments:
  -h, --help            show this help message and exit
  -t <FILE>, --taxlist <FILE>
                        text file containing list of taxa to search for on
                        Genbank
  -a <FILE>, --accessionlist <FILE>
                        text file containing list of accessions to fetch from
                        Genbank
  -m <string>, --marker <string>
                        name of gene to be searched for (put in "" if the
                        search term consists of more than one word
  -G, --Genbank         search Genbank
  -B, --BOLD            search BOLD
  --no-download         search Genbank and just return the number of records
                        found f

Download all available COI sequences for the taxonomic groups specified in the file, one by one. This will download >1.5 M sequences from Genbank so it takes a few hours.

In [ ]:
%%bash

for t in $(cat hexapoda_to_download.txt)
do
    echo -e "\n### Downloading: $t ###\n"
    echo $t > current.txt
    fetch_from_db.py -G -m COI -o $t -t current.txt 
    rm current.txt
done

Ok, now lets download stuff for the remaining groups.

In [13]:
%%file others_to_downlaod.txt
Arachnida
Myriapoda
Mollusca
Nematomorpha
Oniscidea
Vertebrata

Overwriting others_to_downlaod.txt


The Vertebrata download broke a few times and I had to resume it by restarting the script and point it to the file `Vertebrate.accessions.txt` (using the option`-a`) that is produced during the download and that contains the accessions of the records that have not yet been downloaded. Vertebrates are of minor important in this dataset so you can also just skip them alltogether.

In [ ]:
%%bash

for t in $(cat others_to_downlaod.txt)
do
    echo -e "\n### Downloading: $t ###\n"
    echo $t > current.txt
    fetch_from_db.py -G -m COI -o $t -t current.txt 
    rm current.txt
done

Some Fungi.

In [12]:
%%file Fungi_to_download.txt
Cercospora
Didymella
Calonectria
Fusarium
Melampsora

Overwriting Fungi_to_download.txt


In [ ]:
!fetch_from_db.py -G -m COI -o Fungi -t Fungi_to_download.txt

Positive controls.

In [15]:
%%file postives.txt
Astatotilapia calliptera
Triops cancriformis
Comaster audax

Overwriting postives.txt


In [ ]:
!fetch_from_db.py -G -m COI -o positives -t postives.txt

Some weird records that hamper the correct taxonomic assignment for _Harmonia axyridis_ should be excluded.

In [17]:
!head -n 10 to_exclude.accessions.txt

AJ312060.1
AJ313070.1
AY533712.1
AY533715.1
AY533722.1
AY533732.1
AY533736.1
AY533742.1
AY533751.1
AY533753.1


In [5]:
fh=open('to_exclude.accessions.txt','r')

to_ex = []

for l in fh:
    to_ex.append(l.strip())

fh.close()

Format reference database for use with Kraken.

In [53]:
def gb_to_blast(db_in_gb, db_for_blast, to_exclude=[], minlength=200, maxlength=10000, v=0):

    from Bio import SeqIO

    Seqs_new = []
    out = open(db_for_blast,'w')
    ex = []
    total = 0
    
    if to_exclude:
        ex = list(set(to_exclude))
    
    for f in db_in_gb:
        print "processing %s .. " %f,
        Seqs = SeqIO.parse(open(f,'r'), 'genbank')
        count = 0
        ok = 0
        skipped = 0
        excluded = 0
        
        for r in Seqs:
            count+=1
            if r.id in ex or len(r.seq) < minlength or len(r.seq) > maxlength:
                if v:
                    print "\nexcluding %s" %r.id
                excluded+=1
                continue
            taxid = ''
            source=r.features[0]
            for t in source.qualifiers['db_xref']:
                if 'taxon' in t:
                    taxid = t.split(":")[1]
        
            if taxid:    
                organism = source.qualifiers['organism'][0].replace(' ','_')
                r.description = "%s|%s|%s" %(r.id,taxid,organism)
                r.id = r.description
                Seqs_new.append(r)
                ok+=1
            else:
                if v:
                    print "\nWARNING: No taxid detected for record: %s - skipping" %r.id
                skipped+=1
                continue

            if len(Seqs_new) == 1000:
            
                SeqIO.write(Seqs_new, out, 'fasta')
                Seqs_new = []

        SeqIO.write(Seqs_new, out, 'fasta')
    
        print " converted %i / %i sequences (%i excluded, %i skipped).\n" %(ok,count,excluded,skipped)
        total+=ok
    out.close()
    
    print "Done - Converted a total of %i records!" %total

In [54]:
import os

datadir = './'
####################
gbs = []
files = os.listdir(datadir)
for f in sorted(files):
    if f.endswith('gb'):
        gbs.append(f)
        
gb_to_blast(db_in_gb=gbs, db_for_blast='all.blast.fasta', to_exclude=to_ex, minlength=400, maxlength=1000, v=0)

processing Amphiesmenoptera.gb ..   converted 400056 / 421311 sequences (21255 excluded, 0 skipped).

processing Arachnida.gb ..   converted 80859 / 87682 sequences (6823 excluded, 0 skipped).

processing Coleoptera.gb ..   converted 127019 / 136607 sequences (9588 excluded, 0 skipped).

processing Diptera.gb ..   converted 589433 / 600506 sequences (11073 excluded, 0 skipped).

processing Embioptera.gb ..   converted 78 / 105 sequences (27 excluded, 0 skipped).

processing Ephemeroptera.gb ..   converted 10900 / 11230 sequences (330 excluded, 0 skipped).

processing Fungi.gb ..   converted 110 / 163 sequences (53 excluded, 0 skipped).

processing Hymenoptera.gb ..   converted 180363 / 192517 sequences (12154 excluded, 0 skipped).

processing Mecoptera.gb ..   converted 371 / 378 sequences (7 excluded, 0 skipped).

processing Mollusca.gb ..   converted 87255 / 92813 sequences (5558 excluded, 0 skipped).

processing Myriapoda.gb ..   converted 2259 / 2328 sequences (69 excluded, 0 skipp

In [55]:
def gb_to_kraken(db_in_gb, db_for_kraken, to_exclude=[], minlength=200, maxlength=10000, v=0):

    from Bio import SeqIO

    Seqs_new = []
    out = open(db_for_kraken,'w')
    
    total = 0
    ex = []
    
    if to_exclude:
        ex = list(set(to_exclude))
    
    for f in db_in_gb:
        print "processing %s .. " %f,
        Seqs = SeqIO.parse(open(f,'r'), 'genbank')
        count = 0
        ok = 0
        skipped = 0
        excluded = 0
        
        for r in Seqs:
            count+=1
            if r.id in ex or len(r.seq) < minlength or len(r.seq) > maxlength:
                if v:
                    print "\nexcluding %s" %r.id
                excluded+=1
                continue
            taxid = ''
            source=r.features[0]
            for t in source.qualifiers['db_xref']:
                if 'taxon' in t:
                    taxid = t.split(":")[1]
        
            if taxid:        
                r.id = "%s|kraken:taxid|%s" %(r.id,taxid)
                r.description = r.id
                Seqs_new.append(r)
                ok+=1
            else:
                if v:
                    print "\nWARNING: No taxid detected for record: %s - skipping" %r.id
                skipped+=1
                continue

            if len(Seqs_new) == 1000:
            
                SeqIO.write(Seqs_new, out, 'fasta')
                Seqs_new = []

        SeqIO.write(Seqs_new, out, 'fasta')
    
        print " converted %i / %i sequences (%i excluded, %i skipped).\n" %(ok,count,excluded,skipped)
        total+=ok
    out.close()
    
    print "Done - Converted a total of %i records!" %total

In [57]:
import os

datadir = './'
####################
gbs = []
files = os.listdir(datadir)
for f in sorted(files):
    if f.endswith('gb'):
        gbs.append(f)
        
gb_to_kraken(db_in_gb=gbs, db_for_kraken='all.kraken.fasta', to_exclude=to_ex, minlength=400, maxlength=1000, v=0)

processing Amphiesmenoptera.gb ..   converted 400056 / 421311 sequences (21255 excluded, 0 skipped).

processing Arachnida.gb ..   converted 80859 / 87682 sequences (6823 excluded, 0 skipped).

processing Coleoptera.gb ..   converted 127019 / 136607 sequences (9588 excluded, 0 skipped).

processing Diptera.gb ..   converted 589433 / 600506 sequences (11073 excluded, 0 skipped).

processing Embioptera.gb ..   converted 78 / 105 sequences (27 excluded, 0 skipped).

processing Ephemeroptera.gb ..   converted 10900 / 11230 sequences (330 excluded, 0 skipped).

processing Fungi.gb ..   converted 110 / 163 sequences (53 excluded, 0 skipped).

processing Hymenoptera.gb ..   converted 180363 / 192517 sequences (12154 excluded, 0 skipped).

processing Mecoptera.gb ..   converted 371 / 378 sequences (7 excluded, 0 skipped).

processing Mollusca.gb ..   converted 87255 / 92813 sequences (5558 excluded, 0 skipped).

processing Myriapoda.gb ..   converted 2259 / 2328 sequences (69 excluded, 0 skipp

Filter database - only keep sequences that have a reasonable kmer overlap with queries. I use `mirabait` from the MIRA assembler to identify reference sequences that share at least 5 kmers of length 31 with a query. Could be more stringent even, to narrow it down further, but this will do for now.

Get `mirabait` just to do this quickly.

In [ ]:
%%bash
mkdir mira
cd mira
wget https://sourceforge.net/projects/mira-assembler/files/MIRA/stable/mira_4.0.2_linux-gnu_x86_64_static.tar.bz2
tar -xvjf mira_4.0.2_linux-gnu_x86_64_static.tar.bz2
cd ..

In [ ]:
%%bash

mira/mira_4.0.2_linux-gnu_x86_64_static/bin/mirabait \
-n 5 -k 31 \
../taxonomic_assignment_full_NT/GLOBAL/global_centroids.fasta all.kraken.fasta \
kraken_k31n5 > mirabait-kraken.log

mira/mira_4.0.2_linux-gnu_x86_64_static/bin/mirabait \
-n 5 -k 31 \
../taxonomic_assignment_full_NT/GLOBAL/global_centroids.fasta all.blast.fasta \
blast_k31n5 > mirabait-blast.log

`mirabait` is not needed any longer.

In [72]:
!rm -rf mira/

Create blast database (takes about a minute).

In [66]:
%%bash

makeblastdb -in blast_k31n5.fasta -dbtype nucl -out blast_k31n5



Building a new DB, current time: 09/23/2016 23:18:57
New DB name:   blast_k31n5
New DB title:  blast_k31n5.fasta
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1319968 sequences in 47.9384 seconds.


Create Kraken database (takes about 1.5 h using 10 threads and requires about 20G of diskspace).

In [61]:
!kraken-build --download-taxonomy --db kraken_k31n5

--2016-09-23 14:37:35--  ftp://ftp.ncbi.nih.gov/pub/taxonomy/gi_taxid_nucl.dmp.gz
           => ‘gi_taxid_nucl.dmp.gz’
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.13
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/taxonomy ... done.
==> SIZE gi_taxid_nucl.dmp.gz ... 1425122001
==> PASV ... done.    ==> RETR gi_taxid_nucl.dmp.gz ... done.
Length: 1425122001 (1.3G) (unauthoritative)

100%[====================================>] 1,425,122,001 33.7MB/s   in 52s    

2016-09-23 14:38:28 (26.2 MB/s) - ‘gi_taxid_nucl.dmp.gz’ saved [1425122001]

Downloaded GI to taxon map
--2016-09-23 14:38:28--  ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz
           => ‘taxdump.tar.gz’
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.13
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.13|:21... connected.
Logging in

In [62]:
!kraken-build --add-to-library kraken_k31n5.fasta --db kraken_k31n5

Added "kraken_k31n5.fasta" to library (kraken_k31n5)


In [63]:
!kraken-build --build --threads 10 --db kraken_k31n5

Kraken build set to minimize disk writes.
Creating k-mer set (step 1 of 6)...
Found jellyfish v1.1.11
Hash size not specified, using '1001477687'
K-mer set created. [27.318s]
Skipping step 2, no database reduction requested.
Sorting k-mer set (step 3 of 6)...
K-mer set sorted. 49.986s]
Creating GI number to seqID map (step 4 of 6)...
GI number to seqID map created. [7.495s]
Creating seqID to taxID map (step 5 of 6)...
1312753 sequences mapped to taxa. [4.256s]
Setting LCAs in database (step 6 of 6)...
Finished processing 1319968 sequences
Database LCAs set. [1h18m39.563s]
Database construction complete. [Total: 1h27m8.646s]
